In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/deep_learning_ppg')

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import architectures
import network_training
import plotting
import preprocessing

In [3]:
def read_data_from_csv(csv_file_name):
      data = pd.read_csv(csv_file_name)
      x_data = data.iloc[:, :-2].values
      y_data = data.iloc[:, -2].values
      subj = data.iloc[:, -1].values

      return x_data, y_data, subj

In [4]:
dataset_id = 2

if dataset_id == 1:
  csv_file_name = '/content/drive/MyDrive/deep_learning_ppg/data_wesad.csv'
  labels = ('baseline', 'stress', 'amusement', 'meditation')
  avg = False
elif dataset_id == 2:
  csv_file_name = '/content/drive/MyDrive/deep_learning_ppg/data_affectiveroad.csv'
  labels = ('low', 'medium', 'high')
  avg = True

features, targets, subj_data = read_data_from_csv(csv_file_name)
fs = 64

In [ ]:
# Optional downsampling

fs_new = 8

features = preprocessing.resample_signal(features, fs, fs_new)
targets = preprocessing.resample_signal(targets, fs, fs_new)

fs = fs_new

In [5]:
# Use data from one wrist only: 0 - left, 1 - right
wrist_id = 0

features = features[:, wrist_id].reshape(-1, 1)

In [6]:
params = {
    "seed" : 256,
    "learning_rate" : 0.001,
    "weight_decay" : 1e-6,
    "step_size" : 3,
    "gamma" : 0.85,
    "batch_size" : 128,
    "epochs" : 30,
    "num_resblocks" : 3,
    "num_lstm_layers" : 2,
    "num_lstm_units" : 256
}

params["window_size"] = fs * 8
params["overlap"] = params["window_size"] * 3//4

In [ ]:
scaled_data = preprocessing.scale_data(features, targets)

# import sklearn
# scaled_data = sklearn.preprocessing.normalize(features)

sliding_X_data, sliding_y_data = preprocessing.apply_sliding_window(scaled_data, targets, subj_data, params["window_size"], params["overlap"], avg)

X_data = sliding_X_data.astype(np.float32)
y_data = sliding_y_data.astype(np.uint8)

print(X_data.shape, y_data.shape)

In [8]:
params["num_channels"] = X_data.shape[2]
params["num_classes"] = len(labels)

In [9]:
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.model_selection import KFold

def training_loop(X_data, y_data, params, architecture_id, crossvalid = False):
  if crossvalid:
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state = params["seed"])
    data_splits = [(X_data[train_ind], X_data[val_ind], y_data[train_ind], y_data[val_ind]) for train_ind, val_ind in kf.split(X_data)]
  else:
    num_folds = 1
    X_train, X_val, y_train, y_val = network_training.split_data(X_data, y_data, train_size=0.8)
    data_splits = [(X_train, X_val, y_train, y_val)]

  avg_losses_train = []
  f1_scores_train = []
  accuracies_train = []
  recalls_train = []
  precisions_train = []

  avg_losses_val = []
  f1_scores_val = []
  accuracies_val = []
  recalls_val = []
  precisions_val = []

  for fold, (X_train_fold, X_val_fold, y_train_fold, y_val_fold) in enumerate(data_splits):
    print(f"Fold {fold+1}/{num_folds}")

    if architecture_id == 1:
                network = architectures.DeepConvLSTM(params["num_channels"], params["num_classes"], num_layers=params["num_lstm_layers"], hidden_size=params["num_lstm_units"])
    else:
                network = architectures.ResNet(params["num_channels"], params["num_classes"], num_resblocks=params["num_resblocks"])


    if fold == 0: print(network)

    optimizer = torch.optim.Adam(network.parameters(), lr=params["learning_rate"], weight_decay=params["weight_decay"])
    class_weights = preprocessing.get_class_weights(y_data)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=params["step_size"], gamma=params["gamma"])

    trainloader, valloader = network_training.data_loader(X_train_fold, y_train_fold,
                                                                  X_val_fold, y_val_fold, params["batch_size"])

    for epoch in range(params["epochs"]):
      print(f"Epoch {epoch + 1}:")
      avg_loss_train, f1_train, accuracy_train, precision_train, recall_train = \
           network_training.train(trainloader, network, optimizer, criterion, scheduler)
      avg_loss_val, f1_val, accuracy_val, precision_val, recall_val = \
           network_training.valid(valloader, network, criterion)

    avg_losses_train.append(avg_loss_train)
    f1_scores_train.append(f1_train)
    accuracies_train.append(accuracy_train)
    recalls_train.append(recall_train)
    precisions_train.append(precision_train)

    avg_losses_val.append(avg_loss_val)
    f1_scores_val.append(f1_val)
    accuracies_val.append(accuracy_val)
    recalls_val.append(recall_val)
    precisions_val.append(precision_val)

  mean_avg_losses_train = np.mean(avg_losses_train)
  mean_f1_scores_train = np.mean(f1_scores_train)
  mean_accuracies_train = np.mean(accuracies_train)
  mean_recalls_train = np.mean(recalls_train)
  mean_precisions_train = np.mean(precisions_train)

  mean_avg_losses_val = np.mean(avg_losses_val)
  mean_f1_scores_val = np.mean(f1_scores_val)
  mean_accuracies_val = np.mean(accuracies_val)
  mean_recalls_val = np.mean(recalls_val)
  mean_precisions_val = np.mean(precisions_val)

  print(f"""Train: F1 - {mean_f1_scores_train*100:.2f} acc - {mean_accuracies_train*100:.2f}
        prec - {mean_precisions_train*100:.2f} rec - {mean_recalls_train*100:.2f}
        loss - {mean_avg_losses_train:.2f}""")
  print(f"""Valid: F1 - {mean_f1_scores_val*100:.2f} acc -  {mean_accuracies_val*100:.2f}
        prec - {mean_precisions_val*100:.2f}  rec - {mean_recalls_val*100:.2f}
        loss - {mean_avg_losses_val:.2f}""")
  return network

In [ ]:
np.random.seed(params["seed"])
torch.manual_seed(params["seed"])

# LSTM - 1, ResNet - 2
architecture_id = 2
model = training_loop(X_data, y_data, params, architecture_id, crossvalid=False)